# 1. RAG 데이터 생성

## 1.1. (미사용) chunk, splitter 함수 

In [ ]:
# from langchain_community.document_loaders import Docx2txtLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# file_path = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\주택임대차보호법(법률)(제21065호)(20260102).docx'
# loader = Docx2txtLoader(file_path)
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=800, 
#     # chunk_overlap=200,
#     separators=['\n제']
# )
# document_list = loader.load_and_split(text_splitter)
# len(document_list)

In [ ]:
# for i in range(len(document_list)):
#     print(document_list[i].page_content[:50])

## 1.2. (사용) regex 활용 함수 정의 

In [10]:
import re
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document

# 1. 파일 로드
file_path = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\주택임대차보호법(법률)(제21065호)(20260102).docx'
loader = Docx2txtLoader(file_path)
docs = loader.load()

# 원본 텍스트 가져오기
origin_text = docs[0].page_content

def split_by_article_safe(text):
    """
    줄바꿈(\n)과 함께 시작하는 '제N조'만 인식하여 분리합니다.
    문장 중간에 나오는 '제N조'는 무시합니다.
    """
    # [Tip] 첫 번째 조항(제1조)도 패턴에 걸리게 하기 위해, 맨 앞에 줄바꿈을 하나 추가합니다.
    padded_text = "\n" + text.strip()
    
    # 수정된 정규표현식
    # (\n : 줄바꿈으로 시작하고)
    # (제 : '제' 글자가 나오며)
    # (\s* : 공백이 0개 이상 있고)
    # (\d+ : 숫자가 있으며)
    # (\s* : 공백이 0개 이상 있고)
    # (조 : '조' 글자로 끝남)
    pattern = r'(\n제\s*\d+\s*조)'
    
    # 패턴으로 분리
    parts = re.split(pattern, padded_text)
    
    chunks = []
    
    # parts[0]은 첫 조항 나오기 전의 서문(법령 제정일 등)일 수 있습니다.
    if parts and parts[0].strip():
        chunks.append(parts[0].strip())
        
    # split 결과는 [서문, \n제1조, 본문, \n제2조, 본문...] 순서입니다.
    # 따라서 1번 인덱스부터 2개씩 묶어서 처리합니다.
    for i in range(1, len(parts), 2):
        if i + 1 < len(parts):
            title = parts[i].strip()   # 예: 제1조
            body = parts[i+1].strip()  # 예: (목적) ... 내용
            
            combined_text = f"{title}{body}"
            chunks.append(combined_text)
            
    return chunks

# 2. 실행
legal_chunks = split_by_article_safe(origin_text)

In [11]:
for i in range(len(legal_chunks)):
    print(legal_chunks[i][:40])

주택임대차보호법

주택임대차보호법 ( 약칭: 주택임대차법 )

[시행 2
제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에
제2조(적용 범위) 이 법은 주거용 건물(이하 “주택”이라 한다)의 전부
제3조(대항력 등) ① 임대차는 그 등기(登記)가 없는 경우에도 임차인(
제3조의2(보증금의 회수) ① 임차인(제3조제2항 및 제3항의 법인을 포
제3조의3(임차권등기명령) ① 임대차가 끝난 후 보증금이 반환되지 아니한
제3조의4(「민법」에 따른 주택임대차등기의 효력 등) ① 「민법」 제62
제3조의5(경매에 의한 임차권의 소멸) 임차권은 임차주택에 대하여 「민사
제3조의6(확정일자 부여 및 임대차 정보제공 등) ① 제3조의2제2항의 
제3조의7(임대인의 정보 제시 의무) 임대차계약을 체결할 때 임대인은 다
제4조(임대차기간 등) ① 기간을 정하지 아니하거나 2년 미만으로 정한 
제5조삭제 <1989. 12. 30.>
제6조(계약의 갱신) ① 임대인이 임대차기간이 끝나기 6개월 전부터 2개
제6조의2(묵시적 갱신의 경우 계약의 해지) ① 제6조제1항에 따라 계약
제6조의3(계약갱신 요구 등) ① 제6조에도 불구하고 임대인은 임차인이 
제7조(차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차
제7조의2(월차임 전환 시 산정률의 제한) 보증금의 전부 또는 일부를 월
제8조(보증금 중 일정액의 보호) ① 임차인은 보증금 중 일정액을 다른 
제8조의2(주택임대차위원회) ① 제8조에 따라 우선변제를 받을 임차인 및
제9조(주택 임차권의 승계) ① 임차인이 상속인 없이 사망한 경우에는 그
제10조(강행규정) 이 법에 위반된 약정(約定)으로서 임차인에게 불리한 
제10조의2(초과 차임 등의 반환청구) 임차인이 제7조에 따른 증액비율을
제11조(일시사용을 위한 임대차) 이 법은 일시사용하기 위한 임대차임이 
제12조(미등기 전세에의 준용) 주택의 등기를 하지 아니한 전세계약에 관
제13조(「소액사건심판법」의 준용) 임차인이 임대인에 대하여 제

In [52]:
# 제목을 추출하는 함수(exaone3.5) : powershell이나 cmd창에서 ollama pull exaone3.5:2.4b
from langchain_ollama import ChatOllama
def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
        num_predict = 60,
    )
    prompt = f"""다음 주택임대차보호법 조문의 핵심 제목을 50토큰 이내로 끊김 없이, 간결하고 명확하게 추출해 주세요."
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return title
content = """제8조(보증금 중 일정액의 보호) ① 임차인은 보증금 중 일정액을 다른 담보물권자(擔保物權者)보다 우선하여 변제받을 권리가 있다. 이 경우 임차인은 주택에 대한 경매신청의 등기 전에 제3조제1항의 요건을 갖추어야 한다.
② 제1항의 경우에는 제3조의2제4항부터 제6항까지의 규정을 준용한다.
③ 제1항에 따라 우선변제를 받을 임차인 및 보증금 중 일정액의 범위와 기준은 제8조의2에 따른 주택임대차위원회의 심의를 거쳐 대통령령으로 정한다. 다만, 보증금 중 일정액의 범위와 기준은 주택가액(대지의 가액을 포함한다)의 2분의 1을 넘지 못한다.<개정 2009. 5. 8.>
[전문개정 2008. 3. 21.]
"""
print(extract_title_with_llm(content))

**보증금 우선 변제권**  
임차인은 보증금 중 일정액을 담보권자보다 우선 변제받을 권리가 있으며, 주택 경매 전 요건 충족 필요.


In [40]:
def get_law_category():
    """
    주택임대차 RAG 시스템을 위한 카테고리-키워드 가중치 사전
    Returns:
        dict: {카테고리명: {키워드: 가중치}}
    """
    return {
        # 1. 보증금 보호 (가장 치명적인 리스크)
        '보증금_대항력': {
            '보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3,
            '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2,
            '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2,
            '반환보증': 2, 'HUG': 1, '보증보험': 2
        },

        # 2. 계약 갱신 및 거절 (가장 빈번한 분쟁)
        '계약갱신': {
            '계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3,
            '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1,
            '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1
        },

        # 3. 계약 해지 및 종료 (탈출 전략)
        '계약해지': {
            '계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2,
            '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2,
            '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3
        },

        # 4. 차임(월세) 및 보증금 증감
        '임대료_증감': {
            '차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2,
            '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2,
            '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1
        },

        # 5. 유지보수 및 원상회복 (민법 영역)
        '수선_원상회복': {
            '수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2,
            '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3,
            '비용상환': 2, '보존행위': 2, '관리비': 1
        },

        # 6. 전세사기 및 권리 리스크 (독소조항 탐지용)
        '권리_리스크': {
            '전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3,
            '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2,
            '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2,
            '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, 
            '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2
        },

        # 7. 행정 절차 (Rules Index 연결)
        '행정절차': {
            '확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3,
            '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2,
            '임대차계약증서': 2, '전자계약': 2, '신분증': 1
        },

        # 8. 소송 및 분쟁 해결 (최후 수단)
        '분쟁해결': {
            '분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3,
            '판결': 2, '집행권원': 2, '경매': 3, '공매': 2,
            '내용증명': 2, '손해배상': 3, '지연이자': 2
        },
        # [신설] 9. 임차권 승계 및 가족 (제9조 타겟팅)
        '임차권_승계': {
            '임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3,
            '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, 
            '공동상속': 2, '반환청구권': 2
        }
    }

In [41]:
def categorize_content(content, top_k=None):
    """내용 카테고리 분류 - 점수 기반으로 모든 카테고리를 점수 순으로 반환
    Parameters :
    - content : 분류할 텍스트 내용
    - top_k : 상위 몇개까지 카테고리를 반환할지(None이면 모든 카테고리 반환)
    Returns:
    - 카테고리 리스트(점수 높은 순)"""
    category_keywords = get_law_category()
    category_scores = {}
    # 각 카테고리별 점수 계산
    for category, weighted_keywords in category_keywords.items():
        # print(category, weighted_keywords)
        score = 0
        for keyword, weight in weighted_keywords.items():
            #if keyword in content: # content에 keyword 가 포함되어 있는지 여부
            #    score += weight
            count = content.count(keyword) # content에 keyword가 몇번 나오는지
            score += count * weight
        if score > 0:
            category_scores[category] = score
    # print(category_scores)
    # 내림차순 정렬한 카테고리 이름만 추출
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True) # 내림차순
    #print(sorted_categories)
    all_categories = [category[0] for category in sorted_categories]
    # print(all_categories)
    # 매칭되는 카테고리가 없으면 "기타" 반환
    if not all_categories:
        all_categories = ["기타"]
    # top_k가 지정되면 상위 top_k개만, 아니면 전체 반환
    if top_k is not None:
        return all_categories[:top_k]
    else:
        return all_categories

In [49]:
categorize_content("집주인이 반년 만에 보증금 인상을 요구할 수 있습니까?")

['보증금_대항력', '임대료_증감']

In [54]:
# 기타로 분류된 청크
for i in range(len(legal_chunks)):
    if categorize_content(legal_chunks[i])==["기타"]:
        print(legal_chunks[i][:10])

주택임대차보호법


제1조(목적) 이 
제2조(적용 범위)
제5조삭제 <198
제18조(조정위원의
제19조(조정위원의
제28조(비밀유지의
제30조(주택임대차


In [ ]:
# 조항 추출하기
def get_article(content):
    "조항들 추출"
    article_match = re.findall(r'제(\d+)조', content)
    article = list(set(article_match)) # 중복제거
    article.sort(key=lambda x:int(x))
    if article:
        return ', '.join([f"{a}조" for a in article])

In [56]:
import re
from langchain_core.documents import Document

# 1. (최적화) 정규표현식으로 제목 추출 함수
def extract_article_title_regex(content):
    """
    LLM 대신 정규표현식으로 '제N조(제목)' 형태를 빠르게 추출합니다.
    실패 시 첫 줄을 반환합니다.
    """
    # 패턴: 제(숫자)조 + (괄호 안 제목) 또는 그냥 제N조
    # 예: "제3조(대항력 등)" -> "제3조(대항력 등)" 추출
    # 예: "제3조" -> "제3조" 추출
    match = re.match(r'(제\d+조(?:\(.*?\))?)', content.strip())
    if match:
        return match.group(1)
    else:
        # 매칭 안 되면 첫 줄의 앞부분 30자만 반환
        return content.strip().split('\n')[0][:30]

# 2. 조항 번호 추출 함수 (파트너님 코드 활용)
def get_article_numbers(content):
    """본문에 포함된 제N조 번호들을 리스트로 추출"""
    matches = re.findall(r'제(\d+)조', content)
    # 중복 제거 및 정렬
    unique_articles = sorted(list(set(matches)), key=lambda x: int(x))
    if unique_articles:
        return ", ".join([f"{a}조" for a in unique_articles])
    return ""

# 3. LangChain Document 변환 및 메타데이터 주입
documents = []

print("데이터 변환 시작...")

for i, chunk in enumerate(legal_chunks):
    # (1) 카테고리 분석
    categories = categorize_content(chunk, top_k=2) # 상위 2개만
    
    # (2) 제목 추출 (Regex 사용으로 속도 개선)
    title = extract_article_title_regex(chunk) + " " + extract_title_with_llm(chunk)
    
    # (3) 관련 조항 번호 추출
    article_nums = get_article_numbers(chunk)
    
    # (4) 메타데이터 조립
    metadata = {
        "src_title": "주택임대차보호법(법률)(제21065호)(20260102)",
        "source": "주택임대차보호법(법률)(제21065호)(20260102).docx", # 파일명
        "category": categories,     # 리스트 형태 (예: ['보증금_대항력', '임대료_증감'])
        "priority": 1,              # 1순위 (int)
        "chunk_id": f"law_chunk_{i:03d}", # 고유 ID
        "title": title,             # 예: 제3조(대항력 등)
        "article": article_nums     # 예: 3조, 3조의2
    }
    
    # (5) Document 객체 생성
    doc = Document(page_content=chunk, metadata=metadata)
    documents.append(doc)

print(f"변환 완료! 총 {len(documents)}개의 조항이 처리되었습니다.")
print("="*40)

# --- 검증: 제9조, 제10조가 잘 분류되었는지 확인 ---
target_articles = ["제9조", "제10조"]
for doc in documents:
    # 제목에 제9조 또는 제10조가 포함되어 있는지 확인
    for target in target_articles:
        if target in doc.metadata['title']:
            print(f"[{doc.metadata['title']}]")
            print(f" - 분류된 카테고리: {doc.metadata['category']}")
            print(f" - 우선순위: {doc.metadata['priority']}")
            print("-" * 30)

데이터 변환 시작...
변환 완료! 총 43개의 조항이 처리되었습니다.
[제9조(주택 임차권의 승계) **주택 임차권 승계**  
- 상속인 없이 사망 시 사실상 혼인 관계자 승계  
- 상속인 없는 경우, 친족 포함 공동 승계 가능  
- 승계 대상자 반대 시 승계 불가  
- 임대차 채권·채무 승계]
 - 분류된 카테고리: ['임차권_승계']
 - 우선순위: 1
------------------------------
[제10조(강행규정) **임대차보호 강화**  
임차인 불리한 약정 무효]
 - 분류된 카테고리: ['권리_리스크']
 - 우선순위: 1
------------------------------
[제10조 **초과 차임 반환 청구**  
임차인이 증액비율 초과 차임/보증금 지급 시 반환 청구 가능]
 - 분류된 카테고리: ['임대료_증감', '보증금_대항력']
 - 우선순위: 1
------------------------------


In [67]:
import pandas as pd

# csv 백업
def save_documents_to_csv(documents, file_name='chunked.csv'):
    # 1. LangChain Document 객체를 딕셔너리 리스트로 변환
    data_list = []
    
    for doc in documents:
        # 기본 데이터 구조 생성
        row = {
            "chunk_id": doc.metadata.get("chunk_id"),
            "priority": doc.metadata.get("priority"),
            "category": str(doc.metadata.get("category")), # 리스트는 문자열로 변환해야 CSV에 잘 들어감
            "title": doc.metadata.get("title"),
            "article": doc.metadata.get("article"),
            "source": doc.metadata.get("source"),
            "page_content": doc.page_content, # 본문 내용
            # 필요하다면 전체 메타데이터를 백업용으로 한 컬럼에 넣을 수도 있음
            # "metadata_json": doc.metadata 
        }
        data_list.append(row)
    
    # 2. DataFrame 생성
    df = pd.DataFrame(data_list)
    
    # 3. CSV 저장 (한글 깨짐 방지: utf-8-sig)
    import os
    save_path = os.path.join("C:/Users/Admin/Desktop/Project2/1_sourceData/1-2_chunked/", file_name)
        
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"CSV 저장 완료: {save_path}")
    print(f"총 {len(df)}행이 저장되었습니다.")
    
    return df

# 실행
df_result = save_documents_to_csv(documents, '주택임대차보호법.csv')

# 상위 3개 행 미리보기
df_result.head(3)

CSV 저장 완료: C:/Users/Admin/Desktop/Project2/1_sourceData/1-2_chunked/주택임대차보호법.csv
총 43행이 저장되었습니다.


,chunk_id,priority,category,title,article,source,page_content
0,law_chunk_000,1,['기타'],주택임대차보호법 **임대차보호법** \n*임대차 계약 보호 및 임차인 권리 보장*,,주택임대차보호법(법률)(제21065호)(20260102).docx,주택임대차보호법\n\n주택임대차보호법 ( 약칭: 주택임대차법 )\n\n[시행 202...
1,law_chunk_001,1,['기타'],제1조(목적) **주거안정 보장을 위한 임대차 특례**,1조,주택임대차보호법(법률)(제21065호)(20260102).docx,제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에 대한 특례...
2,law_chunk_002,1,['기타'],제2조(적용 범위) **주택임대차보호법 적용 범위** \n주거용 건물 임대차 전반...,2조,주택임대차보호법(법률)(제21065호)(20260102).docx,제2조(적용 범위) 이 법은 주거용 건물(이하 “주택”이라 한다)의 전부 또는 일부...


## 1.3. chunking 전 과정(1.2.) 함수화

In [15]:
import os
import re
import pandas as pd
from langchain_ollama import ChatOllama
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.documents import Document

def get_law_category():
    return {
        '보증금_대항력': {'보증금': 3, '대항력': 3, '우선변제권': 3, '최우선변제': 3, '보증금반환': 3, '전세금': 2, '소액임차인': 2, '확정일자': 2, '전입신고': 2, '점유': 2, '임차권등기': 3, '배당': 2, '반환보증': 2, 'HUG': 1, '보증보험': 2},
        '계약갱신': {'계약갱신': 3, '갱신요구': 3, '갱신거절': 3, '묵시적갱신': 3, '계약연장': 2, '실거주': 3, '존속기간': 2, '2년': 1, '재계약': 2, '갱신청구': 2, '거절사유': 2, '복비': 1},
        '계약해지': {'계약해지': 3, '해지통고': 3, '중도해지': 3, '기간만료': 2, '계약종료': 2, '퇴거': 2, '이사': 1, '명도': 2, '합의해지': 2, '통지': 1, '3개월': 2, '즉시해지': 3},
        '임대료_증감': {'차임': 3, '월세': 2, '증액': 3, '감액': 2, '인상': 2, '5퍼센트': 3, '5%': 3, '20분의 1': 2, '상한': 2, '전환율': 2, '월차임': 2, '경제사정': 1, '부담': 1},
        '수선_원상회복': {'수선': 3, '수리': 3, '원상회복': 3, '파손': 2, '훼손': 2, '누수': 2, '곰팡이': 2, '보일러': 2, '필요비': 3, '유익비': 3, '비용상환': 2, '보존행위': 2, '관리비': 1},
        '권리_리스크': {'전세사기': 3, '깡통전세': 3, '신탁': 3, '근저당': 3, '저당권': 3, '선순위': 3, '가압류': 3, '압류': 3, '국세': 2, '지방세': 2, '체납': 3, '납세증명': 2, '위반건축물': 2, '불법건축물': 2, '특약': 2, '독소조항': 2, '강행규정': 3, '효력이 없다': 3, '무효': 3, '불리한 약정': 3, '편면적 강행규정': 2},
        '행정절차': {'확정일자부여': 3, '동주민센터': 2, '등기소': 2, '인터넷등기소': 3, '수수료': 1, '열람': 2, '제공요청': 2, '이해관계인': 2, '임대차계약증서': 2, '전자계약': 2, '신분증': 1},
        '분쟁해결': {'분쟁조정': 3, '조정위원회': 2, '지급명령': 3, '소송': 3, '판결': 2, '집행권원': 2, '경매': 3, '공매': 2, '내용증명': 2, '손해배상': 3, '지연이자': 2},
        '임차권_승계': {'임차권승계': 3, '승계': 3, '사망': 3, '상속': 3, '상속인': 3, '사실혼': 3, '배우자': 2, '가정공동생활': 3, '2촌': 2, '공동상속': 2, '반환청구권': 2}
    }

def categorize_content(content, top_k=None):
    category_keywords = get_law_category()
    category_scores = {}
    for category, weighted_keywords in category_keywords.items():
        score = 0
        for keyword, weight in weighted_keywords.items():
            count = content.count(keyword)
            score += count * weight
        if score > 0:
            category_scores[category] = score
    
    sorted_categories = sorted(category_scores.items(), key=lambda x:x[1], reverse=True)
    all_categories = [category[0] for category in sorted_categories]
    
    if not all_categories:
        all_categories = ["기타"]
    
    return all_categories[:top_k] if top_k else all_categories

def split_by_article_safe(text):
    padded_text = "\n" + text.strip()
    pattern = r'(\n제\s*\d+\s*조)'
    parts = re.split(pattern, padded_text)
    
    chunks = []
    if parts and parts[0].strip():
        chunks.append(parts[0].strip())
        
    for i in range(1, len(parts), 2):
        if i + 1 < len(parts):
            title = parts[i].strip()
            body = parts[i+1].strip()
            chunks.append(f"{title} {body}")
    return chunks

def extract_article_title_regex(content):
    match = re.match(r'(제\d+조(?:\(.*?\))?)', content.strip())
    return match.group(1) if match else content.strip().split('\n')[0][:30]

def extract_title_with_llm(content):
    "exaone3.5를 사용하여 content의 제목을 추출"
    title_extractor_llm = ChatOllama(
        model = "exaone3.5:2.4b",
        temperature = 0.1,
        num_predict = 60,
    )
    prompt = f"""다음 주택임대차보호법 조문의 핵심 제목을 50토큰 이내로 끊김 없이, 간결하고 명확하게 추출해 주세요. 추출한 제목만 출력합니다."
    조문 : {content}"""
    ai_message = title_extractor_llm.invoke(prompt)
    title = ai_message.content.strip()
    return title

def get_article_numbers(content):
    matches = re.findall(r'제(\d+)조', content)
    unique_articles = sorted(list(set(matches)), key=lambda x: int(x))
    return ", ".join([f"{a}조" for a in unique_articles]) if unique_articles else ""

# 타깃 문서 파일 처리

def process_single_file(file_path, priority, output_dir='./data/1-2_chunked/'):
    """
    파일 하나를 처리하여 Documents 리스트를 반환하고 CSV로 저장합니다.
    [수정사항] 메타데이터 생성 시 None 값이 들어가는 것을 원천 차단합니다.
    """
    
    # 1. 파일 존재 확인 및 경로 정보 추출
    if not os.path.exists(file_path):
        print(f"❌ Error: 파일을 찾을 수 없습니다: {file_path}")
        return []

    file_name = os.path.basename(file_path)
    # 확장자 제거한 파일명 (src_title로 사용됨)
    file_name_no_ext = os.path.splitext(file_name)[0]
    
    # [안전장치] 파일명이 비어있을 경우 방지
    if not file_name_no_ext:
        file_name_no_ext = "Unknown_Source"
    
    print(f"🚀 처리 시작: {file_name} (우선순위: {priority})")

    # 2. 로드
    loader = Docx2txtLoader(file_path)
    docs = loader.load()
    origin_text = docs[0].page_content
    
    # 3. 청킹
    legal_chunks = split_by_article_safe(origin_text)
    print(f"   - 청킹 완료: {len(legal_chunks)}개 조항")
    
    # 4. 메타데이터 입히기 (결측치 방어 로직 적용)
    processed_documents = []
    
    for i, chunk in enumerate(legal_chunks):
        categories = categorize_content(chunk, top_k=None)
        
        # 제목 추출 (None 방지: or "" 추가)
        extracted_title = extract_article_title_regex(chunk)
        llm_title = extract_title_with_llm(chunk) # LLM 사용 시
        # llm_title = "" # LLM 미사용 시 주석 해제하고 위 줄 주석 처리
        
        full_title = f"{extracted_title} {llm_title}".strip()
        
        article_nums = get_article_numbers(chunk)
        
        # [핵심 수정] 메타데이터 조립 시 모든 필드 강제 형변환 및 None 제거
        metadata = {
            "chunk_id": f"{file_name_no_ext}_{i:03d}",
            "priority": int(priority), # 정수 강제 변환
            "category": categories if categories else ["기타"], # 빈 리스트 방지
            "title": str(full_title or "").strip(),     # None -> ""
            "article": str(article_nums or "").strip(), # None -> ""
            "source": str(file_name or "").strip(),     # None -> ""
            
            # 여기가 문제였던 부분입니다! 확실하게 문자열로 변환
            "src_title": str(file_name_no_ext or "").strip() 
        }
        
        # 내용(page_content)도 안전하게 처리
        safe_content = str(chunk or "").strip()
        
        doc = Document(page_content=safe_content, metadata=metadata)
        processed_documents.append(doc)
        
    # 5. CSV 저장
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    df_data = []
    for doc in processed_documents:
        row = doc.metadata.copy()
        row['page_content'] = doc.page_content
        row['category'] = str(row['category']) # 리스트 -> 문자열 변환 (CSV용)
        df_data.append(row)
        
    df = pd.DataFrame(df_data)
    csv_path = os.path.join(output_dir, f"{file_name_no_ext}.csv")
    
    # utf-8-sig로 저장해야 엑셀에서 한글이 안 깨집니다.
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ CSV 저장 완료: {csv_path}")
    print("-" * 40)
    
    return processed_documents

# ==========================================
# 3. 사용코드 (여기서 파일만 바꿔가며 실행)
# ==========================================

# ▼▼▼ 파일 경로와 우선순위 설정 ▼▼▼
target_file_path = r'C:\Users\Admin\Desktop\Project2\1_sourceData\1-1_sourceData\주택임대차보호법(법률)(제21065호)(20260102).docx'
target_priority = 1 # (1:특별법, 2:시행령, 3:규칙, 4:민법)

# 실행 및 변수 할당
law_docs = process_single_file(target_file_path, target_priority)

# 결과 확인 (첫 번째 문서만)
if law_docs:
    print(f"\n[결과 미리보기] 첫 번째 청크:\n{law_docs[0].metadata}")

🚀 처리 시작: 주택임대차보호법(법률)(제21065호)(20260102).docx (우선순위: 1)
   - 청킹 완료: 43개 조항
✅ CSV 저장 완료: ./data/1-2_chunked/주택임대차보호법(법률)(제21065호)(20260102).csv
----------------------------------------

[결과 미리보기] 첫 번째 청크:
{'chunk_id': '주택임대차보호법(법률)(제21065호)(20260102)_000', 'priority': 1, 'category': ['기타'], 'title': '주택임대차보호법 **주택임대차계약 보호**', 'article': '', 'source': '주택임대차보호법(법률)(제21065호)(20260102).docx', 'src_title': '주택임대차보호법(법률)(제21065호)(20260102)'}


In [16]:
law1 = law_docs
law1[0].page_content
law1[0].metadata

{'chunk_id': '주택임대차보호법(법률)(제21065호)(20260102)_000',
 'priority': 1,
 'category': ['기타'],
 'title': '주택임대차보호법 **주택임대차계약 보호**',
 'article': '',
 'source': '주택임대차보호법(법률)(제21065호)(20260102).docx',
 'src_title': '주택임대차보호법(법률)(제21065호)(20260102)'}

In [14]:
law2 = law_docs
law2[0].page_content
law2[0].metadata

{'chunk_id': '주택임대차보호법 시행령(대통령령)(제35947호)(20260102)_000',
 'priority': 2,
 'category': ['기타'],
 'title': '주택임대차보호법 시행령 2026년 주택임대차보호법 시행령 시행',
 'article': '',
 'source': '주택임대차보호법 시행령(대통령령)(제35947호)(20260102).docx',
 'src_title': '주택임대차보호법 시행령(대통령령)(제35947호)(20260102)'}

In [11]:
law4 = law_docs
law4[0].page_content
law4[0].metadata

{'chunk_id': '민법(법률)(제20432호)(20260101)_000',
 'priority': 4,
 'category': ['기타'],
 'title': '민법 채권의 효력 발생 및 유지',
 'article': '',
 'source': '민법(법률)(제20432호)(20260101).docx',
 'src_title': '민법(법률)(제20432호)(20260101)'}

## 1.4. Embedding

In [17]:
import os
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

# ==========================================
# 1. 환경 설정 및 API 키 로드
# ==========================================
# .env 파일 재로드 (혹시 모를 키 변경 반영)
load_dotenv(override=True)

pc_api_key = os.getenv("PINECONE_API_KEY").strip()
up_api_key = os.getenv("UPSTAGE_API_KEY").strip()

print(f"🔑 Pinecone Key 확인: {pc_api_key[:5]}**********")

# ==========================================
# 2. 데이터 병합 (List Merge)
# ==========================================
# 각 변수가 존재하는지 확인 후 합치기 (안전장치)
all_docs = []

# 변수가 정의되어 있는지 확인 (globals() 활용) 또는 직접 리스트 더하기
# 사용자가 law1, law2, law4를 가지고 있다고 가정
try:
    if 'law1' in globals() and law1: all_docs.extend(law1)
    if 'law2' in globals() and law2: all_docs.extend(law2)
    if 'law4' in globals() and law4: all_docs.extend(law4)
    # 시행규칙(law3) 등 다른 변수가 있다면 여기에 추가:
    # if 'law3' in globals() and law3: all_docs.extend(law3)
    
except NameError:
    print("⚠️ 주의: law1, law2, law4 변수 중 일부가 정의되지 않았습니다. 앞선 셀을 실행했는지 확인하세요.")

if not all_docs:
    print("❌ 병합할 데이터가 없습니다. process_single_file 함수를 먼저 실행해주세요.")
else:
    print(f"📦 총 {len(all_docs)}개의 법령 청크를 병합했습니다.")

    # ==========================================
    # 3. Pinecone 인덱스 준비
    # ==========================================
    pc = Pinecone(api_key=pc_api_key)
    index_name = "law-index"
    
    # 임베딩 모델 설정 (Upstage Solar: 4096 차원)
    embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

    # 인덱스가 없으면 생성
    if index_name not in pc.list_indexes().names():
        print(f"⚙️ '{index_name}' 인덱스를 새로 생성합니다...")
        pc.create_index(
            name=index_name,
            dimension=4096, # Solar Embedding 차원 필수 일치
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print("   - 인덱스 생성 완료")
    else:
        print(f"ℹ️ '{index_name}' 인덱스가 이미 존재합니다. 데이터를 추가합니다.")

    # ==========================================
    # 4. 임베딩 및 업로드 (Upsert)
    # ==========================================
    print("🚀 Pinecone 업로드 시작 (데이터 양에 따라 1~2분 소요)...")
    
    try:
        database = PineconeVectorStore.from_documents(
            documents=all_docs,
            embedding=embedding,
            index_name=index_name,
            pinecone_api_key=pc_api_key # 명시적 전달
        )
        print("🎉 [미션 성공] 모든 법령 데이터가 Pinecone에 저장되었습니다!")
        
    except Exception as e:
        print(f"🔥 업로드 중 에러 발생: {e}")
        print("Tip: 메타데이터에 여전히 None이 있거나 API Key 문제일 수 있습니다.")

c:\Users\Admin\Desktop\Project2\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔑 Pinecone Key 확인: pcsk_**********
📦 총 131개의 법령 청크를 병합했습니다.
⚙️ 'law-index' 인덱스를 새로 생성합니다...
   - 인덱스 생성 완료
🚀 Pinecone 업로드 시작 (데이터 양에 따라 1~2분 소요)...
🎉 [미션 성공] 모든 법령 데이터가 Pinecone에 저장되었습니다!


# 2. LLM 튜닝

## 2.1. 키워드 사전

In [ ]:

# 업로드한 벡터 DB를 가져오기
database = PineconeVectorStore(
    embedding=embedding,
    index_name=index_name
)

# 간단한 검색 테스트
query = "보증금 증액은 몇 퍼센트까지 가능해?"
results = database.similarity_search(query, k=3)
for doc in results:
    print(f"[{doc.metadata['title']}] {doc.page_content[:100]}...")



[제8조 차임 증액 제한 및 기간 규정] 제8조 (차임 등 증액청구의 기준 등) ① 법 제7조에 따른 차임이나 보증금(이하 “차임등”이라 한다)의 증액청구는 약정한 차임등의 20분의 1의 금액을 초과하지 못한다.

② 제...
[제7조 차임 및 보증금 증감청구권] 제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에...
[제10조 초과 차임 반환 청구 규정] 제10조 의2(초과 차임 등의 반환청구) 임차인이 제7조에 따른 증액비율을 초과하여 차임 또는 보증금을 지급하거나 제7조의2에 따른 월차임 산정률을 초과하여 차임을 지급한 경우에는...


In [25]:
# 질문 표준화를 위한 키워드 사전 정의
keyword_dict = [
    # 계약 주체 및 대상
    "[집주인, 건물주, 주인집, 임대업자] -> [임대인]",
    "[세입자, 월세 사는 사람, 전세 사는 사람] -> [임차인]",
    "[집, 방, 빌라, 아파트, 오피스텔] -> [임차주택, 주거용 건물]",
    "[계약서, 집문서, 종이, 서류] -> [임대차계약증서]",
    # 금전 관련
    "[월세, 방세, 다달이 내는 돈] -> [차임]",
    "[보증금, 전세금, 맡긴 돈, 떼인 돈] -> [임차보증금]",
    "[월세 올리기, 인상, 더 달라고 함] -> [차임의 증액]",
    "[월세 깎기, 내리기, 할인] -> [차임의 감액]",
    "[돈 먼저 받기, 순위, 안전장치, 돌려받기] -> [우선변제권, 최우선변제권]",
    "[복비, 수수료, 중개비, 중개료] -> [중개보수]",
    # 계약 상태 및 변화
    "[이사 가기, 들어오기, 짐 빼기] -> [주택의 인도]",
    "[전입신고, 주소 옮기기, 주소지 이전] -> [주민등록]",
    "[연장하기, 한 번 더 살기, 2+2, 갱신] -> [계약갱신요구권]",
    "[자동 연장, 연락 없음] -> [묵시적 갱신]",
    "[나가라고 함, 쫓겨남, 그만 살기] -> [계약의 해지, 갱신 거절]",
    "[집주인 바뀜, 매매] -> [임대인 지위 승계]",
    # 수리 및 사고
    "[집 고치기, 수리, 곰팡이, 물 샘, 고장] -> [수선의무, 필요비]",
    "[깨끗이 치우기, 원래대로 해놓기, 청소] -> [원상회복의무]",
    "[깡통전세, 사기, 경매 넘어감] -> [전세피해, 권리 리스크]",
    "[집주인 사망, 자식에게 상속] -> [임차권 승계]"
]

In [31]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

"exaone3.5 사용 답변 테스트"
response_llm = ChatOllama(model = "exaone3.5:2.4b")
prompt = ChatPromptTemplate.from_template(f"""우리의 사전을 참고해서 사용자의 질문을 변경하고, 변경된 질문만 리턴해 주세요.
만약 변경할 필요가 없을 경우, 사용자의 질문을 변경하지 않아도 되니 입력된 질문만 그대로 리턴해 주세요.
사전 : {keyword_dict}
질문 : {{question}}""")
keyword_chain = prompt | response_llm | StrOutputParser()
query = "집주인이 월세를 올려 달래요. 제가 어렵다고 하니까 저보고 다음 달까지 나가라고 하네요. 계약서에는 그런 얘기가 없는데 이렇게 쫓겨나도 되나요?"
normalized_query = keyword_chain.invoke({"question":query})
normalized_query

'**변경된 질문:**\n\n임대차계약증서에 명시된 계약 조건 없이 임대인이 월세를 인상하고 계약 해지를 요구하는 상황에서의 법적 권리와 책임은 어떻게 되나요? 특히 계약서에 그러한 조항이 없을 경우, 임차인이 다음 달까지 머무를 수 있는 권리는 무엇인가요?'

2.2. 

In [35]:
print("query:", categorize_content(query))
print("normalized_query:", categorize_content(normalized_query))

query: ['임대료_증감']
normalized_query: ['임대료_증감', '행정절차']


## 2.2. (미사용) Retriever

In [ ]:
# # Retriever 생성 (category중 하나라도 포함되는 있는 10개 chunk 추출)
# retriever = database.as_retriever(
#     search_kwargs = {
#         "k":10,
#         "filter":{"category":{"$in":categorize_content(normalized_query)}}
#     }
# )
# docs = retriever.invoke(normalized_query)
# print("관련 문서")
# for i, doc in enumerate(docs):
#     chunk_id = doc.metadata.get('chunk_id')
#     title = doc.metadata.get('title')
#     article = doc.metadata.get('article')
#     print(f"{i+1}.{chunk_id} {article} - {title}")

관련 문서
1.주택임대차보호법(법률)(제21065호)(20260102)_014 6조, 7조 - 제6조 **계약갱신 요구권**
2.주택임대차보호법(법률)(제21065호)(20260102)_012 6조 - 제6조 임대차 계약 갱신 보장
3.주택임대차보호법(법률)(제21065호)(20260102)_015 7조 - 제7조 차임 및 보증금 증감청구권
4.민법(법률)(제20432호)(20260101)_025 640조 - 제640조 임대차 계약 해지 조건
5.주택임대차보호법 시행령(대통령령)(제35947호)(20260102)_009 2조, 7조, 8조, 15조 - 제8조 차임 증액 제한 및 기간 규정
6.민법(법률)(제20432호)(20260101)_013 628조 - 제628조 차임증감청구권
7.주택임대차보호법(법률)(제21065호)(20260102)_021 7조, 10조 - 제10조 초과 차임 반환 청구 규정
8.민법(법률)(제20432호)(20260101)_012 627조 - 제627조 ## 핵심 제목:  **임차인의 감액청구 및 해지권**
9.주택임대차보호법(법률)(제21065호)(20260102)_009 3조, 5조, 6조, 108조, 109조 - 제3조 임대인 정보 제공 의무
10.주택임대차보호법 시행령(대통령령)(제35947호)(20260102)_006 3조, 5조, 6조, 12조 - 제5조 주택임대차보호법 시행령 제5조: 이해관계인의 정의


## 2.3. Retriever + Rerank

In [65]:
import os
import cohere
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv(override=True)
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

def rerank_retriever(query, pinecone_store, k_fetch=30, k_final=10, score_threshold=0.25):
    """
    [수정됨] 법률 RAG 전용 검색 파이프라인 (Threshold 필터링 추가)
    
    Args:
        query (str): 사용자 질문
        pinecone_store: LangChain PineconeVectorStore 객체
        k_fetch (int): 1차 검색 후보 개수 (기본 30)
        k_final (int): 최종 반환할 최대 개수 (기본 10)
        score_threshold (float): 이 점수 미만은 과감히 버림 (기본 0.25)
    """
    
    # 1. Pinecone 1차 검색 (Vector Similarity)
    print(f"🔍 1차 검색 (Pinecone): '{query}' 관련 문서 {k_fetch}개 탐색 중...")
    initial_docs = pinecone_store.similarity_search(query, k=k_fetch)
    
    if not initial_docs:
        print("⚠️ 1차 검색 결과가 없습니다.")
        return []

    # 2. Cohere AI Re-ranking
    # 문서 내용 추출
    docs_content = [doc.page_content for doc in initial_docs]
    
    co = cohere.Client(api_key=COHERE_API_KEY)
    
    try:
        rerank_results = co.rerank(
            model="rerank-multilingual-v3.0",
            query=query,
            documents=docs_content,
            top_n=k_fetch # 일단 1차 검색 결과를 다 평가해봅니다.
        )
    except Exception as e:
        print(f"❌ Cohere Rerank 에러: {e}")
        return initial_docs[:k_final] # 에러 시 그냥 1차 검색 결과 반환

    # 3. 점수 필터링 및 메타데이터 주입
    relevant_docs = []
    
    print(f"📊 Rerank 점수 분석 (Threshold: {score_threshold}):")
    
    for r in rerank_results.results:
        current_score = r.relevance_score
        
        # [핵심 수정] 기준 점수 미만이면 통과(Skip)
        if current_score < score_threshold:
            continue
            
        # 인덱스로 원본 문서 찾기
        doc = initial_docs[r.index]
        # 디버깅용 점수 기록
        doc.metadata['relevance_score'] = current_score
        relevant_docs.append(doc)
        
        # 상위 몇 개만 로그 출력
        if len(relevant_docs) <= 5:
            print(f"   - [Pass] {current_score:.4f} : {doc.metadata.get('title')}")

    # 4. 필터링 후 남은 게 없으면?
    if not relevant_docs:
        print("⚠️ Rerank 결과, 기준 점수(0.25)를 넘는 관련 문서가 없습니다.")
        # 강제로라도 1등 하나를 반환할지, 빈 리스트를 줄지 결정
        # 여기서는 빈 리스트를 반환하여 챗봇이 "관련 법령을 찾지 못했습니다"라고 말하게 유도
        return []

    # 5. Priority 기반 정렬 (Legal Hierarchy Sorting)
    # 1순위(특별법) -> ... -> 99(기타) 순서로 정렬
    final_docs = sorted(
        relevant_docs, 
        key=lambda x: int(x.metadata.get('priority', 99))
    )
    
    # 최종 개수 제한 (k_final)
    final_docs = final_docs[:k_final]
    
    print(f"✅ 최종 선별 완료: {len(final_docs)}개 문서 반환")
    return final_docs

# 실행 테스트
reranked_docs = rerank_retriever(normalized_query, database) #fetch 30, final 10, threshold 0.25
print(f"검색된 문서 개수: {len(reranked_docs)}")
for i, doc in enumerate(reranked_docs):
    r = doc.metadata.get('relevance_score')
    p = doc.metadata.get('priority')
    title = doc.metadata.get('title')
    print(f"[{i+1}] 관련도{r:.4f} (우선순위: {p}) {title}")

🔍 1차 검색 (Pinecone): '질문: 임차인이 임대인에게 월세 인상을 요구하고, 임차인이 어려움을 표현하자 임대인이 다음 달까지 이사를 요청하셨습니다. 계약에 명시된 내용이 없음에도 불구하고 이러한 상황에서 계약 해지가 정당할까요?' 관련 문서 30개 탐색 중...
📊 Rerank 점수 분석 (Threshold: 0.25):
   - [Pass] 0.5826 : 제628조 차임증감청구권
   - [Pass] 0.4898 : 제7조 차임 및 보증금 증감청구권
   - [Pass] 0.3733 : 제8조 차임 증액 제한 및 기간 규정
   - [Pass] 0.2799 : 제626조 임차인의 상환청구권
   - [Pass] 0.2672 : 제10조 초과 차임 반환 청구 규정
✅ 최종 선별 완료: 5개 문서 반환
검색된 문서 개수: 5
[1] 관련도0.4898 (우선순위: 1.0) 제7조 차임 및 보증금 증감청구권
[2] 관련도0.2672 (우선순위: 1.0) 제10조 초과 차임 반환 청구 규정
[3] 관련도0.3733 (우선순위: 2.0) 제8조 차임 증액 제한 및 기간 규정
[4] 관련도0.5826 (우선순위: 4.0) 제628조 차임증감청구권
[5] 관련도0.2799 (우선순위: 4.0) 제626조 임차인의 상환청구권


## 2.4. Langchain 답변 생성

In [67]:
print(len(reranked_docs))
print(reranked_docs[0].metadata['src_title'])
print(reranked_docs[0].page_content.replace("\n\n", "\n"))

5
주택임대차보호법(법률)(제21065호)(20260102)
제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있다. 이 경우 증액청구는 임대차계약 또는 약정한 차임이나 보증금의 증액이 있은 후 1년 이내에는 하지 못한다. <개정 2020. 7. 31.>
② 제1항에 따른 증액청구는 약정한 차임이나 보증금의 20분의 1의 금액을 초과하지 못한다. 다만, 특별시ㆍ광역시ㆍ특별자치시ㆍ도 및 특별자치도는 관할 구역 내의 지역별 임대차 시장 여건 등을 고려하여 본문의 범위에서 증액청구의 상한을 조례로 달리 정할 수 있다.<신설 2020. 7. 31.>
[전문개정 2008. 3. 21.]


In [ ]:
def format_legal_docs(docs):
    """
    Reranking된 문서 리스트를 받아 
    1) 웹 화면 표시용 HTML (Bootstrap 스타일)
    2) LLM 입력용 텍스트
    두 가지 형태로 변환하여 반환.
    """
    
    display_html = ""  # 웹 화면용 (Bootstrap)
    llm_context = ""   # AI 입력용 (Plain Text)
    
    # Bootstrap 카드 시작 (아코디언이나 스크롤 박스로 감싸도 좋습니다)
    display_html += '<div class="legal-docs-container mb-3">'
    display_html += '<h6 class="text-primary fw-bold mb-2">📚 관련 법령 및 판례</h6>'
    
    for i, doc in enumerate(docs):
        # 1. 데이터 추출
        src_title = doc.metadata.get('src_title', '출처 미상')
        title = doc.metadata.get('title', '')
        # HTML에서는 줄바꿈을 <br>로 표현하거나 CSS로 처리해야 함
        content = doc.page_content.replace('\n\n','\n').strip()
        priority = doc.metadata.get('priority', 99)
        
        # 2. LLM 입력용 텍스트 구성 (깔끔하게)
        llm_context += f"참고문서 {i+1}.\n출처: {src_title}\n조항: {content}\n\n"
        
        # 3. 웹 화면용 HTML 구성 (Bootstrap 5 Card Style)
        # badge 색상: 우선순위가 높을수록(1) 눈에 띄게
        badge_color = "bg-danger" if priority == 1 else "bg-secondary"
        
        display_html += f"""
        <div class="card mb-2 shadow-sm" style="font-size: 0.9rem;">
            <div class="card-header d-flex justify-content-between align-items-center py-2">
                <span class="fw-bold text-dark">{title}</span>
                <span class="badge {badge_color}">{src_title}</span>
            </div>
            <div class="card-body py-2 bg-light">
                <p class="card-text text-secondary" style="white-space: pre-line;">{content}</p>
            </div>
        </div>
        """
        
    display_html += '</div>'
    
    return {
        "display_html": display_html,
        "llm_context": llm_context
    }

In [ ]:
# reranked_docs는 이미 구했다고 가정
formatted_data = format_legal_docs(reranked_docs)

# 1) LLM에게 줄 텍스트 확인
print("=== [LLM Input Context] ===")
print(formatted_data['llm_context'])

# 2) 웹에 뿌려질 HTML 확인(여기선 텍스트 형태)
print("\n=== [Web HTML Output] ===")
print(formatted_data['display_html'][:200] + "...") # 앞부분만 출력

=== [LLM Input Context] ===
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있다. 이 경우 증액청구는 임대차계약 또는 약정한 차임이나 보증금의 증액이 있은 후 1년 이내에는 하지 못한다. <개정 2020. 7. 31.>
② 제1항에 따른 증액청구는 약정한 차임이나 보증금의 20분의 1의 금액을 초과하지 못한다. 다만, 특별시ㆍ광역시ㆍ특별자치시ㆍ도 및 특별자치도는 관할 구역 내의 지역별 임대차 시장 여건 등을 고려하여 본문의 범위에서 증액청구의 상한을 조례로 달리 정할 수 있다.<신설 2020. 7. 31.>
[전문개정 2008. 3. 21.]

참고문서 2.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제10조 의2(초과 차임 등의 반환청구) 임차인이 제7조에 따른 증액비율을 초과하여 차임 또는 보증금을 지급하거나 제7조의2에 따른 월차임 산정률을 초과하여 차임을 지급한 경우에는 초과 지급된 차임 또는 보증금 상당금액의 반환을 청구할 수 있다.
[본조신설 2013. 8. 13.]

참고문서 3.
출처: 주택임대차보호법 시행령(대통령령)(제35947호)(20260102)
조항: 제8조 (차임 등 증액청구의 기준 등) ① 법 제7조에 따른 차임이나 보증금(이하 “차임등”이라 한다)의 증액청구는 약정한 차임등의 20분의 1의 금액을 초과하지 못한다.
② 제1항에 따른 증액청구는 임대차계약 또는 약정한 차임등의 증액이 있은 후 1년 이내에는 하지 못한다.
[전문개정 2008. 8. 21.]
[제2조에서 이동, 종전 제8조는 제15조로 이동 <2013. 12. 30.>]

참고문서 4.
출처: 민법(법률)(제20432호)(20260101)
조항: 제628조 (차임증감청

In [75]:
import os
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드 (OpenAI API Key 등)
# from dotenv import load_dotenv
# load_dotenv()

def generate_legal_advice(normalized_query, context_text):
    """
    사용자 입력(질문 또는 계약서 조항)과 검색된 법률 조항(Context)을 바탕으로 AI 답변을 생성합니다.
    입력의 성격에 따라 질의응답 또는 법적 효력 검토를 수행합니다.
    Args:
        normalized_query (str): 정제된 사용자 질문 또는 계약서 텍스트
        context_text (str): format_legal_docs 함수에서 나온 'llm_context'
    Returns:
        str: AI의 최종 답변
    """
    
    # 1. [디버깅용] 입력값 서버 로그 출력
    print("=" * 40)
    print(f"📢 [User Input]: {normalized_query}")
    print("-" * 40)
    print(f"📚 [Retrieved Context]:\n{context_text[:500]} ... (생략) ...") 
    print("=" * 40)

    # 2. 프롬프트 템플릿 정의 (입력 유형에 따른 유연한 대응)
    system_prompt = """
    당신은 대한민국 부동산 법률(주택임대차보호법, 민법 등)에 정통한 'AI 법률 비서'입니다.
    사용자는 '궁금한 점을 질문'할 수도 있고, '계약서의 특정 조항(특약)'을 입력하여 검토를 요청할 수도 있습니다.
    아래 [참고 법령]을 철저히 준수하여 분석하세요.

    [지시사항]
    1. 근거 기반 분석: 반드시 제공된 [참고 법령]의 내용에 기반하여 답변하세요. 법적 근거가 없다면 솔직히 모른다고 하세요.
    2. 법적 위계 준수: 주택임대차보호법(특별법) > 민법(일반법) > 판례 순으로 우선순위를 두어 적용하세요.    
    3. 입력 유형에 따른 결론 제시 (가장 중요)**:
        - (A) 사용자가 '질문'을 한 경우**: 
            서두에 명확한 결론(가능/불가능/조건부 가능)을 먼저 제시하고 이유를 설명하세요.
        - (B) 사용자가 '계약서 조항/특약'을 입력한 경우**: 
            해당 조항이 **'법적으로 유효한지(유효/무효)'** 또는 **'임차인에게 불리한지'**를 먼저 판정하세요.
            (예: "이 조항은 주택임대차보호법 제10조에 위반되어 무효일 가능성이 높습니다.")

    4. 상세 근거 제시: "주택임대차보호법 제6조에 따르면..."과 같이 근거가 되는 조항이나 판례를 명시하세요.
    
    5. 🚨 독소 조항 및 위험 경고: 
        - 사용자가 입력한 계약 내용이 **'편면적 강행규정'**(임차인에게 불리한 약정은 효력이 없다)에 해당한다면, 반드시 **[주의사항]** 섹션을 만들어 강력하게 경고하세요.
        - 보증금 미반환 위험, 대항력 상실, 원상회복 과다 청구 등의 리스크를 구체적으로 지적하세요.

    6. 면책 조항 답변 마지막에는 항상 "본 답변은 법률적 참고용이며, 법적 효력은 없습니다. 정확한 판단을 위해 법률 전문가와 상담하시기 바랍니다."라는 문구를 포함하세요.

    [참고 법령]
    {context}
    """

    user_prompt = "사용자 입력: {question}"

    # 3. LangChain 체인 구성
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", user_prompt),
    ])

    # 모델 설정
    llm = ChatOllama(
        model="exaone3.5:2.4b", # 로컬 모델 사용 시
        # model="gpt-4o",       # 상용 서비스 배포 시 변경
        temperature=0.1,        # 분석의 일관성을 위해 낮게 설정
    )

    # 체인 연결
    chain = prompt | llm | StrOutputParser()

    # 4. 실행 및 결과 반환
    try:
        response = chain.invoke({
            "context": context_text,
            "question": normalized_query
        })
        return response
    
    except Exception as e:
        error_msg = f"❌ 답변 생성 중 오류 발생: {str(e)}"
        print(error_msg)
        return "죄송합니다. 법률 정보를 분석하는 중 문제가 발생했습니다. 입력하신 내용이 너무 길거나 복잡하지 않은지 확인해 주세요."

In [ ]:
# 테스트 실행 (주피터 노트북용)

# Case 1: 질문일 때
query1 = "집주인이 보증금을 안 돌려주는데 이사 가도 돼?"
norm_query1 = keyword_chain.invoke({"question":query1})
# Case 2: 계약서 조항일 때
query2 = "특약사항: 임차인은 1년 뒤 무조건 퇴거하며, 이를 어길 시 위약금 500만원을 배상한다."
norm_query2 = keyword_chain.invoke({"question":query2})

# 실제 테스트 (둘 중 하나 선택해서 normalized_query에 넣고 실행해보세요)
ai_answer1 = generate_legal_advice(
    normalized_query=norm_query1, 
    context_text=formatted_data['llm_context'] # 앞서 만든 포맷팅된 컨텍스트
)
ai_answer2 = generate_legal_advice(
    normalized_query=norm_query2, 
    context_text=formatted_data['llm_context'] # 앞서 만든 포맷팅된 컨텍스트
)

📢 [User Input]: 질문: 임차인이 임대차 계약을 갱신할 권리가 있을 때 집주인이 임대료를 감액해야 하는 상황에서 이사를 가도 될까요?
----------------------------------------
📚 [Retrieved Context]:
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있다. 이 경우 증액청구는 임대차계약 또는 약정한 차임이나 보증금의 증액이 있은 후 1년 이내에는 하지 못한다. <개정 2020. 7. 31.>
② 제1항에 따른 증액청구는 약정한 차임이나 보증금의 20분의 1의 금액을 초과하지 못한다. 다만, 특별시ㆍ광역시ㆍ특별자치시ㆍ도 및 특별자치도는 관할 구역 내의 지역별 임대차 시장 여건 등을 고려하여 본문의 범위에서 증액청구의 상한을 조례로 달리 정할 수 있다.<신설 2020. 7. 31.>
[전문개정 2008. 3. 21.]

참고문서 2.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제10조 의2(초과 차임 등의 ... (생략) ...
📢 [User Input]: 질문: 계약 조건: 임차인은 계약 후 1년 후에 무조건 계약을 종료해야 하며, 계약 위반 시에는 위약금으로 500만원을 지급해야 한다.
----------------------------------------
📚 [Retrieved Context]:
참고문서 1.
출처: 주택임대차보호법(법률)(제21065호)(20260102)
조항: 제7조 (차임 등의 증감청구권) ① 당사자는 약정한 차임이나 보증금이 임차주택에 관한 조세, 공과금, 그 밖의 부담의 증감이나 경제사정의 변동으로 인하여 적절하지 아니하게 된 때에는 장래에 대하여 그 증감을 청구할 수 있다. 이 경우 증액청구는 

In [79]:
print(norm_query1)
print("\n[AI 답변]:\n")
print(ai_answer1)

질문: 임차인이 임대차 계약을 갱신할 권리가 있을 때 집주인이 임대료를 감액해야 하는 상황에서 이사를 가도 될까요?

[AI 답변]:

**결론**: 가능합니다.

**이유 및 상세 근거**:

주택임대차보호법 제7조에 따르면, 임차인은 임대차 계약 기간 중에 차임(임대료)의 증감을 청구할 수 있는 권리가 있습니다. 특히, 경제사정이나 임차주택에 관한 공과금 등의 변동으로 인해 약정된 차임이 적절하지 않게 되었을 경우, 임차인은 계약 갱신과 함께 차임의 증액 또는 감액을 청구할 수 있습니다. 이는 주택임대차보호법 제7조 제1항에 명시되어 있습니다.

- **증액청구**: 임차인이 차임을 증액하려는 경우, 증액 비율은 약정된 차임의 20분의 1을 초과할 수 없습니다 (주택임대차보호법 제7조 제2항).
- **감액청구**: 반대로 임대인이 임차인에게 차임을 감액하도록 청구할 수 있는 상황에서도 임차인은 이사를 통해 계약을 종료하고 새로운 임대차 계약을 체결할 수 있습니다. 주택임대차보호법 제7조 제1항은 경제사정 변동 등으로 인해 차임이 적절하지 않게 된 경우 임차인이 차임 감액을 청구할 수 있음을 명시하고 있습니다.

**주의사항**:
- **감액 후 계약 갱신**: 임대인이 차임을 감액한 후에도 임차인이 계약 갱신을 원할 경우, 감액된 차임 수준을 유지하거나 그 이하로 계약을 연장할 수 있습니다. 하지만 임대인이 감액된 차임 수준을 유지하지 않고 더 높은 차임을 요구할 경우, 임차인은 감액된 차임 수준을 기준으로 계약 갱신을 거절하거나 새로운 임대인을 찾아 이사를 진행할 수 있습니다.
- **법적 효력**: 감액된 차임 수준을 기준으로 계약을 갱신하거나 이사를 결정할 때는 해당 조항이 법적으로 유효한지 확인하는 것이 중요합니다. 만약 감액 조항이 법적으로 문제가 있다면, 임차인은 이사를 통해 법적 보호를 받을 수 있습니다.

**본 답변은 법률적 참고용이며, 정확한 판단을 위해 법률 전문가와 상담하시기 바랍니다.**


In [80]:
print(norm_query2)
print("\n[AI 답변]:\n")
print(ai_answer2)

질문: 계약 조건: 임차인은 계약 후 1년 후에 무조건 계약을 종료해야 하며, 계약 위반 시에는 위약금으로 500만원을 지급해야 한다.

[AI 답변]:

### 답변

**결론**: **불가능**

**이유**:
주택임대차보호법 제7조 및 제10조의2에 따르면, 임차인의 계약 종료 조건과 위약금 지급 조항에 대한 분석이 필요합니다.

1. **계약 종료 조건**:
   - **주택임대차보호법 제7조**에 따르면, 임차인은 계약 기간 중 차임 등의 증감청구권을 가지며, 이는 계약 기간 동안 자유롭게 청구할 수 있는 권리입니다. 따라서 계약 후 1년 후 무조건 계약을 종료해야 한다는 조항은 법적으로 제한적입니다. 주택임대차보호법은 임차인에게 일정 기간 동안 계약을 유지할 권리를 보장하고 있어, 계약 종료를 강제하는 조항은 법적으로 제한적일 수 있습니다.

2. **위약금 지급 조항**:
   - **주택임대차보호법 제10조의2**에 따르면, 임차인이 계약 기간 중 차임을 초과 지급하거나 월차임 산정률을 초과하여 차임을 지급한 경우 초과 지급된 금액의 반환을 청구할 수 있습니다. 그러나 계약 위반 시 위약금을 무조건적으로 지급해야 한다는 조항은 명확한 법적 근거가 부족합니다. 주택임대차보호법은 주로 차임의 적정성과 관련된 규정을 두고 있으며, 위약금 지급에 대한 강제 조항은 명시되어 있지 않습니다.

**상세 근거**:
- **주택임대차보호법 제7조**: 임차인은 차임 등의 증감청구권을 가지며, 이는 계약 기간 동안 자유롭게 행사할 수 있는 권리입니다. 따라서 계약 종료를 무조건적으로 강제하는 조항은 법적으로 제한적입니다.
- **주택임대차보호법 제10조의2**: 초과 차임 반환 청구 조항이 있으나, 위약금 지급에 대한 강제 조항은 명시되어 있지 않습니다.

**독소 조항 및 위험 경고**:
- **편면적 강행규정**: 계약 조항 중 계약 종료를 무조건적으로 강제하는 부분은 편면적 강행규정에 해당할 가능성이 높습니다. 이는 임차인에게 불리한 조건으로, 법적으로 효

In [ ]:
# 프론트엔드용

# from django.http import JsonResponse

# def chat_api(request):
#     user_query = request.POST.get('query')
    
#     # ... (검색 및 Reranking 로직 수행) ...
#     # reranked_docs = legal_hybrid_retrieval(user_query, ...)
    
#     # 포맷팅 함수 호출
#     formatted_data = format_legal_docs(reranked_docs)
    
#     # LLM 호출 (formatted_data['llm_context'] 사용)
#     # ai_response = llm_chain.invoke({"context": formatted_data['llm_context'], ...})
    
#     # 프론트엔드로 응답 (HTML은 화면에 바로 그리기 용도)
#     return JsonResponse({
#         "status": "success",
#         "answer": ai_response,              # AI의 답변 텍스트
#         "references": formatted_data['display_html'] # 법령 카드 HTML
#     })